# DATA LOADER - Загрузка CSV и сохранение в Parquet
Этот notebook загружает данные из CSV файлов и сохраняет их в формате Parquet для быстрой загрузки

In [1]:
# %% ИМПОРТЫ
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Импорты загружены")

Импорты загружены


In [2]:
# %% КОНФИГУРАЦИЯ ПУТЕЙ
class DataConfig:
    # CSV файлы (источники)
    TRAIN_CSV = "CLTV_UL_TRAIN_MART.csv"
    PROD_CSV = "CLTV_UL_PROD_SNAPSHOT.csv"
    # CHURN_CSV = "../churn_result_msb.csv"
    
    # Parquet файлы (целевые)
    DATA_DIR = Path("data")
    TRAIN_PARQUET = DATA_DIR / "train_data.parquet"
    PROD_PARQUET = DATA_DIR / "prod_data.parquet"
    # CHURN_PARQUET = DATA_DIR / "churn_data.parquet"
    
    @classmethod
    def ensure_directories(cls):
        cls.DATA_DIR.mkdir(parents=True, exist_ok=True)

DataConfig.ensure_directories()
print(f"Директория для данных: {DataConfig.DATA_DIR}")

Директория для данных: data


In [3]:
# %% ФУНКЦИЯ ЗАГРУЗКИ CSV
def load_csv_file(file_path, description="Файл"):
    """
    Загрузка CSV файла с обработкой различных форматов
    """
    if not os.path.exists(file_path):
        print(f"⚠️  Файл не найден: {file_path}")
        return pd.DataFrame()
    
    print(f"📂 Загрузка {description}: {file_path}")
    
    try:
        # Попытка с разделителем '|' и кодировкой windows-1251
        df = pd.read_csv(file_path, sep='|', encoding="windows-1251", thousands=',')
        print(f"   ✓ Загружено (формат: sep='|', encoding='windows-1251')")
        return df
    except Exception as e1:
        try:
            # Попытка с разделителем ','
            df = pd.read_csv(file_path, sep=',')
            print(f"   ✓ Загружено (формат: sep=',')")
            return df
        except Exception as e2:
            try:
                # Попытка с разделителем '\t'
                df = pd.read_csv(file_path, sep='\t')
                print(f"   ✓ Загружено (формат: sep='\\t')")
                return df
            except Exception as e3:
                print(f"   ✗ Ошибка загрузки: {e3}")
                return pd.DataFrame()

In [4]:
# %% ЗАГРУЗКА TRAIN DATA
print("="*70)
print("ЗАГРУЗКА ОБУЧАЮЩИХ ДАННЫХ")
print("="*70)

train_df = load_csv_file(DataConfig.TRAIN_CSV, "Обучающие данные")

if not train_df.empty:
    print(f"\n📊 Статистика:")
    print(f"   Записей: {len(train_df):,}")
    print(f"   Столбцов: {len(train_df.columns)}")
    print(f"   Клиентов: {train_df['CLIENT_ID'].nunique():,}")
    print(f"   Сегменты: {sorted(train_df['SEGMENT_ID'].unique())}")
    print(f"\n   Столбцы: {list(train_df.columns)}")
    
    # Проверка пропусков
    missing = train_df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠️  Пропущенные значения:")
        print(missing[missing > 0])
else:
    print("⚠️  Обучающие данные не загружены!")

ЗАГРУЗКА ОБУЧАЮЩИХ ДАННЫХ
📂 Загрузка Обучающие данные: CLTV_UL_TRAIN_MART.csv
   ✓ Загружено (формат: sep='|', encoding='windows-1251')

📊 Статистика:
   Записей: 4,052,063
   Столбцов: 23
   Клиентов: 233,474
   Сегменты: [1022, 1023, 1026, 1027, 1028, 1040]

   Столбцы: ['CLIENT_ID', 'MONTH_END', 'SEGMENT_ID', 'QUALITY_CODE', 'SUBJECT_KIND_ID', 'EC_SECTOR_ID', 'MARGIN', 'MARGIN_LAG1', 'MARGIN_LAG2', 'MARGIN_LAG3', 'MARGIN_AVG_1M_LAG', 'MARGIN_AVG_2M_LAG', 'MARGIN_AVG_3M_LAG', 'MARGIN_AVG_6M_LAG', 'MARGIN_AVG_12M_LAG', 'MARGIN_STDDEV_12M_LAG', 'MARGIN_GROWTH_RATE_3M', 'MONTH_OF_YEAR', 'QUARTER_OF_YEAR', 'TENURE_MONTHS', 'SEGMENT_AVG_MARGIN', 'SEGMENT_MEDIAN_MARGIN', 'TARGET_NEXT_MARGIN']

⚠️  Пропущенные значения:
MARGIN_LAG1               34354
MARGIN_LAG2               72983
MARGIN_LAG3              115858
MARGIN_AVG_1M_LAG         34354
MARGIN_AVG_2M_LAG         34354
MARGIN_AVG_3M_LAG         34354
MARGIN_AVG_6M_LAG         34354
MARGIN_AVG_12M_LAG        34354
MARGIN_STDDEV_12M_L

In [5]:
# %% ЗАГРУЗКА PROD DATA
print("\n" + "="*70)
print("ЗАГРУЗКА ПРОДАКШН ДАННЫХ")
print("="*70)

prod_df = load_csv_file(DataConfig.PROD_CSV, "Продакшн данные")

if not prod_df.empty:
    print(f"\n📊 Статистика:")
    print(f"   Записей: {len(prod_df):,}")
    print(f"   Столбцов: {len(prod_df.columns)}")
    if 'CLIENT_ID' in prod_df.columns:
        print(f"   Клиентов: {prod_df['CLIENT_ID'].nunique():,}")
    if 'SEGMENT_ID' in prod_df.columns:
        print(f"   Сегменты: {sorted(prod_df['SEGMENT_ID'].unique())}")
    print(f"\n   Столбцы: {list(prod_df.columns)}")
else:
    print("⚠️  Продакшн данные не загружены!")


ЗАГРУЗКА ПРОДАКШН ДАННЫХ
📂 Загрузка Продакшн данные: CLTV_UL_PROD_SNAPSHOT.csv
   ✓ Загружено (формат: sep='|', encoding='windows-1251')

📊 Статистика:
   Записей: 358,787
   Столбцов: 23
   Клиентов: 358,787
   Сегменты: [1022, 1023, 1026, 1027, 1040]

   Столбцы: ['CLIENT_ID', 'MONTH_END', 'SEGMENT_ID', 'QUALITY_CODE', 'SUBJECT_KIND_ID', 'EC_SECTOR_ID', 'MARGIN', 'MARGIN_LAG1', 'MARGIN_LAG2', 'MARGIN_LAG3', 'MARGIN_AVG_1M_LAG', 'MARGIN_AVG_2M_LAG', 'MARGIN_AVG_3M_LAG', 'MARGIN_AVG_6M_LAG', 'MARGIN_AVG_12M_LAG', 'MARGIN_STDDEV_12M_LAG', 'MARGIN_GROWTH_RATE_3M', 'MONTH_OF_YEAR', 'QUARTER_OF_YEAR', 'TENURE_MONTHS', 'SEGMENT_AVG_MARGIN', 'SEGMENT_MEDIAN_MARGIN', 'FORECAST_START_DATE']


In [6]:
# # %% ЗАГРУЗКА CHURN DATA
# print("\n" + "="*70)
# print("ЗАГРУЗКА CHURN ДАННЫХ")
# print("="*70)

# churn_df = load_csv_file(DataConfig.CHURN_CSV, "Churn данные")

# if not churn_df.empty:
#     print(f"\n📊 Статистика:")
#     print(f"   Записей: {len(churn_df):,}")
#     print(f"   Столбцов: {len(churn_df.columns)}")
#     print(f"\n   Столбцы: {list(churn_df.columns)}")
    
#     # Нормализация названий столбцов
#     if 'churn_probability' in churn_df.columns:
#         churn_df = churn_df.rename(columns={'churn_probability': 'CHURN_PROB_3M'})
#         print("   ✓ Переименован столбец 'churn_probability' → 'CHURN_PROB_3M'")
    
#     churn_df.columns = churn_df.columns.str.upper()
#     print(f"   ✓ Столбцы приведены к верхнему регистру")
# else:
#     print("⚠️  Churn данные не загружены!")

In [7]:
# %% СОХРАНЕНИЕ В PARQUET
print("\n" + "="*70)
print("СОХРАНЕНИЕ В PARQUET")
print("="*70)

saved_files = []

# Сохранение train
if not train_df.empty:
    print(f"\n💾 Сохранение обучающих данных...")
    train_df.to_parquet(DataConfig.TRAIN_PARQUET, index=False, compression='snappy')
    file_size = os.path.getsize(DataConfig.TRAIN_PARQUET) / (1024 * 1024)  # MB
    print(f"   ✓ Сохранено: {DataConfig.TRAIN_PARQUET}")
    print(f"   ✓ Размер: {file_size:.2f} MB")
    saved_files.append('train')

# Сохранение prod
if not prod_df.empty:
    print(f"\n💾 Сохранение продакшн данных...")
    prod_df.to_parquet(DataConfig.PROD_PARQUET, index=False, compression='snappy')
    file_size = os.path.getsize(DataConfig.PROD_PARQUET) / (1024 * 1024)  # MB
    print(f"   ✓ Сохранено: {DataConfig.PROD_PARQUET}")
    print(f"   ✓ Размер: {file_size:.2f} MB")
    saved_files.append('prod')

# # Сохранение churn
# if not churn_df.empty:
#     print(f"\n💾 Сохранение churn данных...")
#     churn_df.to_parquet(DataConfig.CHURN_PARQUET, index=False, compression='snappy')
#     file_size = os.path.getsize(DataConfig.CHURN_PARQUET) / (1024 * 1024)  # MB
#     print(f"   ✓ Сохранено: {DataConfig.CHURN_PARQUET}")
#     print(f"   ✓ Размер: {file_size:.2f} MB")
#     saved_files.append('churn')

print(f"\n{'='*70}")
print(f"✅ ЗАГРУЗКА ЗАВЕРШЕНА")
print(f"{'='*70}")
print(f"Сохранено файлов: {len(saved_files)} из 3")
print(f"Готовые файлы: {', '.join(saved_files)}")
print(f"\nДля использования в основном notebook загружайте данные из:")
print(f"  - {DataConfig.TRAIN_PARQUET}")
print(f"  - {DataConfig.PROD_PARQUET}")
# print(f"  - {DataConfig.CHURN_PARQUET}")


СОХРАНЕНИЕ В PARQUET

💾 Сохранение обучающих данных...
   ✓ Сохранено: data\train_data.parquet
   ✓ Размер: 349.20 MB

💾 Сохранение продакшн данных...
   ✓ Сохранено: data\prod_data.parquet
   ✓ Размер: 25.86 MB

✅ ЗАГРУЗКА ЗАВЕРШЕНА
Сохранено файлов: 2 из 3
Готовые файлы: train, prod

Для использования в основном notebook загружайте данные из:
  - data\train_data.parquet
  - data\prod_data.parquet


In [8]:
# %% ПРОВЕРКА ЗАГРУЗКИ ИЗ PARQUET (опционально)
print("\n" + "="*70)
print("ТЕСТ: ПРОВЕРКА ЗАГРУЗКИ ИЗ PARQUET")
print("="*70)

if DataConfig.TRAIN_PARQUET.exists():
    test_train = pd.read_parquet(DataConfig.TRAIN_PARQUET)
    print(f"\n✓ Train загружен: {len(test_train):,} записей")
    print(f"  Первые 5 строк:")
    print(test_train.head())

if DataConfig.PROD_PARQUET.exists():
    test_prod = pd.read_parquet(DataConfig.PROD_PARQUET)
    print(f"\n✓ Prod загружен: {len(test_prod):,} записей")

# if DataConfig.CHURN_PARQUET.exists():
#     test_churn = pd.read_parquet(DataConfig.CHURN_PARQUET)
#     print(f"\n✓ Churn загружен: {len(test_churn):,} записей")

print("\n" + "="*70)
print("✅ ВСЕ ФАЙЛЫ ГОТОВЫ К ИСПОЛЬЗОВАНИЮ!")
print("="*70)


ТЕСТ: ПРОВЕРКА ЗАГРУЗКИ ИЗ PARQUET

✓ Train загружен: 4,052,063 записей
  Первые 5 строк:
   CLIENT_ID                MONTH_END  SEGMENT_ID QUALITY_CODE  \
0       1049  2024-01-31 00:00:00.000        1023           БГ   
1       1049  2024-02-29 00:00:00.000        1023           БГ   
2       1049  2024-03-31 00:00:00.000        1023           БГ   
3       1049  2024-04-30 00:00:00.000        1023           БГ   
4       1049  2024-05-31 00:00:00.000        1023           БГ   

  SUBJECT_KIND_ID  EC_SECTOR_ID        MARGIN   MARGIN_LAG1   MARGIN_LAG2  \
0              ЮЛ          1041  3.849652e+06  2.622380e+07  1.052012e+06   
1              ЮЛ          1041  2.758478e+06  3.849652e+06  2.622380e+07   
2              ЮЛ          1041  3.380613e+06  2.758478e+06  3.849652e+06   
3              ЮЛ          1041  2.407339e+06  3.380613e+06  2.758478e+06   
4              ЮЛ          1041  1.204042e+06  2.407339e+06  3.380613e+06   

    MARGIN_LAG3  ...  MARGIN_AVG_6M_LAG  MARGIN_A